In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D, Embedding
from keras.layers import Input, Reshape, Dot, Add
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
# from keras.utils.vis_utils import plot_model
import keras
import keras.backend as K
from data_handle import *
from gensim_wrapper import *
from utils import *
import gensim
import random
import numpy as np
import tensorflow as tf
seed_val = 1000
random.seed(seed_val)
np.random.seed(seed_val)
tf.set_random_seed(seed_val)

In [158]:
def create_model(num_tokens, seq_length, embed_size, n_units, n_consonant, n_vowels):
    int_words_input = Input(shape=(seq_length, ), dtype='int32')
    embedding = Embedding(input_dim=num_tokens, output_dim=embed_size, input_length=seq_length)(int_words_input)

    _, state_h = GRU(n_units, return_sequences=True, return_state=True, name="first_gru")(embedding)
    
    consonant_decoder_inputs = Input(shape=(None, n_consonant), name="target_consonant")
    consonant_decoder_gru = GRU(n_units, return_sequences=True, return_state=True,  name="consonant_decoder_gru")
    consonant_decoder_outputs, _= consonant_decoder_gru(consonant_decoder_inputs, initial_state=state_h)
    
    vowel_decoder_inputs = Input(shape=(None, n_vowels), name="vowel_input")
    vowel_decoder_gru = GRU(n_units, return_sequences=True, return_state=True, name="vowl_decoder_gru")
    vowel_decoder_outputs, _= vowel_decoder_gru(vowel_decoder_inputs, initial_state=state_h)
#     print(vowel_decoder_outputs.shape, consonant_decoder_outputs.shape)
#     x = Concatenate(axis=1)([vowel_decoder_outputs, consonant_decoder_outputs])
#     print(x.shape)
    consonant_decoder_dense = Dense(n_consonant, activation='softmax', name="consonant_output")
    consonant_decoder_outputs = consonant_decoder_dense(consonant_decoder_outputs)
    
    vowel_decoder_dense = Dense(n_vowels, activation='softmax', name="vowel_output")
    vowel_decoder_outputs = vowel_decoder_dense(vowel_decoder_outputs)
    
    main_model = Model([root_word_input, consonant_decoder_inputs, vowel_decoder_inputs], [consonant_decoder_outputs, vowel_decoder_outputs])
    
    encoder_model = Model(root_word_input, state_h)
    
    decoder_state_input_h = Input(shape=(n_units,))
    
    consonant_decoder_outputs, state_h= consonant_decoder_gru(consonant_decoder_inputs, initial_state=decoder_state_input_h)
    consonant_decoder_outputs = consonant_decoder_dense(consonant_decoder_outputs)
    
    vowel_decoder_outputs, state_h= vowel_decoder_gru(vowel_decoder_inputs, initial_state=decoder_state_input_h)
    vowel_decoder_outputs = vowel_decoder_dense(vowel_decoder_outputs)
    
    decoder_model = Model([consonant_decoder_inputs, vowel_decoder_inputs, decoder_state_input_h], [consonant_decoder_outputs, vowel_decoder_outputs, state_h])

    return main_model, encoder_model, decoder_model
    

In [159]:
words = read_file()
vocab, word2int, int2word = build_vocab(words)
int_words = words_to_ints(word2int, words)
word2freq = get_frequency(words, word2int, int2word)
char2int, int2char, char2tup, tup2char, n_consonant, n_vowels = build_charset()
ns_unigrams = ns_sample(word2freq, word2int, int2word, .75)
n_chars = 11 + 2 
n_features = len(char2int)
batch_size = 120
embed_size = 128
skip_window = 5
seq_len = 50
embeddings = np.load('results/char_embedding.npy')

In [160]:
def generate(int_words, int2word, char2tup, embeddings, seq_length, batch_size,n_chars, n_cons, n_vows):
    embed_size = embeddings.shape[1]
    ci = 0
    while True:
#         batch_inputs = np.ndarray((batch_size, seq_length, embed_size))
        batch_inputs = np.ndarray((batch_size, seq_length), dtype=np.int32)
        batch_cons_output = np.ndarray((batch_size, n_chars, n_cons))
        batch_vows_output = np.ndarray((batch_size, n_chars, n_vows))
        for i in range(batch_size):
            seq, ci = get_context_words(int_words, ci, ci+seq_length + 1)
            ci = ci+1
            target = seq[-1]
            seq = seq[:seq_length]
#             seq_embeds = embeddings[seq]
            con, vow = word2vec_seperated(char2tup, int2word[target], n_chars, n_cons, n_vows)
            batch_cons_output[i] = con
            batch_vows_output[i] = vow
            batch_inputs[i] = seq
        yield batch_inputs, batch_vows_output


In [161]:
gen = generate(int_words, int2word, char2tup, embeddings, seq_len, batch_size, n_chars, n_consonant, n_vowels)

In [165]:
model = create_model(287564, 50, 128, 128, 40, 10)
adam = keras.optimizers.Nadam(0.001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])
model.summary()

(?, 50, 128)
(?, ?, 128)
(?, ?, 128)
(?, 50, 10)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_38 (InputLayer)        (None, 50)                0         
_________________________________________________________________
embedding_35 (Embedding)     (None, 50, 128)           36808192  
_________________________________________________________________
first_gru (GRU)              (None, 50, 128)           98688     
_________________________________________________________________
vowel_decoder_gru (GRU)      [(None, 50, 128), (None,  98688     
_________________________________________________________________
vowel_output (Dense)         (None, 50, 10)            1290      
Total params: 37,006,858
Trainable params: 37,006,858
Non-trainable params: 0
_________________________________________________________________


In [166]:
n_batches = len(vocab) // batch_size
history = model.fit_generator(gen, steps_per_epoch=n_batches, epochs = 3)

Epoch 1/3


ValueError: Error when checking target: expected vowel_output to have shape (50, 10) but got array with shape (13, 10)